<a href="https://colab.research.google.com/github/MangoGrove/MangoGrove.github.io/blob/main/HWK02/CSE_40657_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP HW2: Parsing
*Last Updated: Sep 25 3:28 PM*

For **Google Colab**: Runtime > Change runtime type > Hardware accelerator > GPU

Name: Brisny Rodriguez Flores

In [45]:
!pip install jdc
import jdc

In [2]:
!wget -nc https://raw.githubusercontent.com/aarsri/nlp_hw2/refs/heads/main/utils.py
!wget -nc https://raw.githubusercontent.com/aarsri/nlp_hw2/refs/heads/main/trees.py

--2025-10-03 15:41:12--  https://raw.githubusercontent.com/aarsri/nlp_hw2/refs/heads/main/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 975 [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]     975  --.-KB/s    in 0s      

2025-10-03 15:41:13 (32.8 MB/s) - ‘utils.py’ saved [975/975]

--2025-10-03 15:41:13--  https://raw.githubusercontent.com/aarsri/nlp_hw2/refs/heads/main/trees.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9860 (9.6K) [text/plain]
Saving to: ‘trees.py’

trees.py   

In [3]:
!git clone https://github.com/aarsri/nlp_hw2.git

Cloning into 'nlp_hw2'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 62 (delta 28), reused 12 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 55.67 KiB | 904.00 KiB/s, done.
Resolving deltas: 100% (28/28), done.


## Part 1: Bi-LSTM POS Tagger
In this part, you will build a part-of-speech tagger using the bidirectional LSTM architecture.

In [4]:
from utils import Vocab, read_pos_file

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import random
import pickle
import nltk
from nltk.corpus import treebank

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

nltk.download('treebank')
brown = list(treebank.tagged_sents())

Using device: cpu


[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


**[4 points]** Implement a **bidirectional** LSTM with embedding size 128 and hidden size 256 using torch.nn.LSTM with appropriate parameters set. Remember that PyTorch's cross entropy loss includes softmax.

In [6]:
class BiLSTMTagger(nn.Module):
	def __init__(self, data, embedding_dim, hidden_dim):
		super().__init__()
		self.words = Vocab()
		self.tags = Vocab()


		print(self.words)
		self.words.add('<UNK>')
		self.tags.add('<UNK>')


		for sent in data:
			for word, tag in sent:
				self.words.add(word)
				self.tags.add(tag)

		"""TODO: Populate self.words and self.tags as two vocabulary objects using the Vocab class in utils.py.
		This will allow you to easily numberize and denumberize the word vocabulary as well as the tagset.
		Make sure to add <UNK> self.words."""
		self.embedding_dim = embedding_dim
		self.hidden_dim = hidden_dim
		"""	TODO: Initialize layers."""
		# embedding
		self.embedding = nn.Embedding(len(self.words), embedding_dim)
		# lstm
		self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
		# dropout
		self.dropout = nn.Dropout(0.2)
		# W_out
		self.W_out = nn.Linear(hidden_dim * 2, len(self.tags))
		#raise NotImplementedError

In [7]:
%%add_to BiLSTMTagger
def forward(self, sentence):
	"""	TODO: Pass the sentence through the layers of the model.
		* Because we are using the built-in LSTM, we can pass in an entire sentence rather than iterating through the tokens.
		* IMPORTANT: Because we are dealing with a full sentence now, we have to do minor reshaping.
			* Before passing the embeddings into the LSTM, we have to do `embeddings.view(len(sentence), 1, -1)`
			* Before passing the LSTM output into dropout, we have to do `lstm_out.view(len(sentence), -1)`
		* Return the output scores from the model (pre-softmax). This will be of shape: len(sentence) x total number of tags, meaning each row corresponds to a word, and the values in each row are the scores for all possible POS tags for that word."""
	embedding = self.embedding(sentence)
	embedding = embedding.view(len(sentence), 1, -1) #before sending off the embeddings
	lstm_out, _ = self.lstm(embedding.view(len(sentence), 1, -1))
	lstm_out = lstm_out.view(len(sentence), -1) #before sending off the lstm_output
	drop_out = self.dropout(lstm_out.view(len(sentence), -1))
	scores = self.W_out(drop_out) #pre soft max
	return scores

	#raise NotImplementedError

In [8]:
%%add_to BiLSTMTagger
def predict(self, scores):
	"""	TODO: Return the most likely tag sequence.
		* When the dim argument is provided, torch.argmax(input, dim) returns a tensor containing the indices of the maximum values along that specified dimension.
		* Since each row of scores corresponds to a different word, and each column corresponds to a different tag, specificy dim=1 (take max along columns)."""

	return torch.argmax(scores, dim=1)

	#raise NotImplementedError

**[3 points]** Write **training** and **evaluation** procedures. These should be similar to HW1 Part 2 and 3.
- For debugging purposes only, shorten the training data by commenting out the train_sents += brown line. Accuracy will be high because the train set is otherwise a combination of ATIS and BROWN, while the test set is ATIS. Commenting out this line will still have the model trained and tested on ATIS, yielding a high accuracy.  However, it will not be generalized well enough to do the free response.


In [9]:

%%add_to BiLSTMTagger
"""	TODO: This function is very similar to fit() from HW1."""
def fit(self, data, lr=0.01, epochs=5):
	# 1. Initialize the optimizer. Use `torch.optim.Adam` with `self.parameters()` and `lr`.
	optimizer = torch.optim.Adam(self.parameters(), lr)
	# 2. Set a loss function variable to `nn.CrossEntropyLoss()`. It includes softmax.
	loss_function = nn.CrossEntropyLoss()
	# 3. Loop through the specified number of epochs.
	for epoch in range(epochs):
		curr_time = time.time()
		#	 1. Put the model into training mode using `self.train()`.
		self.train()
		#	 2. Shuffle the training data using random.shuffle().
		random.shuffle(data)
		#	 3. Initialize variables to keep track of the total loss (`total_loss`) and the total number of tokens (`total_tokens`).
		total_loss = 0
		total_tokens = 0
		#	 4. Loop over each sentence in the training data.
		for sentence in data:
			#1. Produce a numberized sequence of the words in the sentence. Make words lowercase first, and convert the sequence to a tensor using something like: `torch.tensor(idxs, dtype=torch.long)`.
			#sentence.lower()
			#idxs = [self.words.numberize(word.lower) for word in sentence]
			#print(sentence)
			palabras = (word.lower() for word, tag in sentence)
			idxs = [self.words.numberize(word) for word in palabras]

			#print("hello")

			num_sequence = torch.tensor (idxs, dtype=torch.long)
	 		#2. Prepare the target labels using something like: `targets = torch.tensor([self.tags.numberize(t) for t in tags], dtype=torch.long)`
			taggela = [tag for word, tag in sentence]

			targets = torch.tensor([self.tags.numberize(t) for t in taggela], dtype=torch.long)

			#print("tags")
	 		#3. Call `self.zero_grad()` to clear any accumulated gradients from the previous update.
			self.zero_grad()
			#4. Pass the prepared sequence into the model by doing `self(sentence)` to obtain scores. This automatically calls forward().
			scores = self(num_sequence)
			#5. Calculate loss, passing in the output scores and the true target labels.
			loss = loss_function(scores, targets)
			#6. Call `loss.backward()` to compute gradients.
			loss.backward()
			#7. Apply gradient clipping to prevent exploding gradients. Use `torch.nn.utils.clip_grad_norm_()` with `self.parameters()` and a `max_norm` of 5.0.
			torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=5.0)
			#8. Call `optimizer.step()` to update the model parameters using the computed gradients.
			optimizer.step()
			#9. Add `loss.item() * len(targets)` to `total_loss`.
			total_loss += loss.item() * len(targets)
			#10. Add `len(targets)` to `total_tokens`.
			total_tokens += len(targets)
			#print("for loop 2 ")
		new_time = time.time()
	 	#5. Compute the average loss per token by dividing `total_loss / total_tokens`.
		avg_loss = total_loss/total_tokens
		#6. For debugging, it will be helpful to print the average loss per token and the runtime after each epoch. Average loss per token should always decrease epoch to epoch.
		print(f"avg loss per token: {avg_loss};		runtime: {new_time-curr_time}")


	#raise NotImplementedError

In [10]:
%%add_to BiLSTMTagger
"""     TODO: Iterating over the sentences in the data, calculate POS tagging accuracy.
                * Use `self.eval()` and `with torch.no_grad()` so that the model is not trained during evaluation.
                * Prepare the sequence and target labels as in fit().
                * Use self.predict() to get the predicted tags, and then check if it matches the real next character found in the data.
                * Divide the total correct predictions by the total number of tokens to get the final accuracy."""
def evaluate(self, data):
  total_correct = 0
  total_tokens = 0
  self.eval()
  with torch.no_grad():
    for sentence in data:
      idxs = [self.words.numberize(word.lower()) for word, tag in sentence] # prepped like before
      num_sequence = torch.tensor (idxs, dtype=torch.long)

      tags = [tag for word, tag in sentence]
      target = torch.tensor([self.tags.numberize(t) for t in tags], dtype=torch.long)

      predicted = self.predict(self(num_sequence))

      #print predicted tag
      denumber_predicted: list[str] = []
      for i in predicted:
        denumber_predicted.append(self.tags.denumberize(i))
      print(denumber_predicted)
      print(sentence)

      total_correct += (predicted == target).sum().item()
      total_tokens += len(target)

    return total_correct / total_tokens

**[1 point]** Report **loss per epoch** and **accuracy**. For full credit, accuracy must be at least 92% on val.pos and test.pos. Include your **saved model** in the submission files.
- Because the dataset is small, scores from randomly initialized runs can have greater variance. If you're close to this score (e.g., 90%), there is likely not a bug and instead you had an unlucky run. You can run it again if you have time or comment that you believe the code is correct.


In [ ]:
from os import read
import utils
def main():
	train = read_pos_file("/content/nlp_hw2/data/train.pos")
	val = read_pos_file("/content/nlp_hw2/data/val.pos")
	test = read_pos_file("/content/nlp_hw2/data/test.pos")

	model = BiLSTMTagger(train, embedding_dim=128, hidden_dim=256)
	print(f"Is '<UNK>' in model.words.sym2num? {'<UNK>' in model.words.sym2num}")
	if '<UNK>' in model.words.sym2num:
		print(f"Index of '<UNK>': {model.words.sym2num['<UNK>']}")
	print(f"Size of word vocabulary: {len(model.words.sym2num)}")
	print(f"Size of tag vocabulary: {len(model.tags.sym2num)}")
	print("--- End Debugging Vocab in main ---\n")

	model.fit(train, lr=0.01, epochs=5)
	print(model.evaluate(val))
	print()
	print(model.evaluate(test))
	torch.save(model, "my_model.pt")

  #raise NotImplementedError

main()

"""TODO: (reference HW1 part3.py)
	  * Use read_pos_file() from utils.py to read train.pos, val.pos, and test.pos.
	  * Initialize the model with training data, embedding dim 128, and hidden dim 256.
	  * Train the model, calling fit(), on the training data.
	  * Test the model, calling evaluate(), on the validation and test data.
	  * Predict outputs for the first ten examples in test.pos.
	  * Remove all instances of `raise NotImplementedError`!"""

Is '<UNK>' in model.words.sym2num? True
Index of '<UNK>': 0
Size of word vocabulary: 352
Size of tag vocabulary: 34
--- End Debugging Vocab in main ---

avg loss per token: 0.537896096710341;		runtime: 3.944688320159912
avg loss per token: 0.12891171719809197;		runtime: 3.901350736618042
avg loss per token: 0.056285242120767166;		runtime: 4.449092388153076
avg loss per token: 0.032563992935914866;		runtime: 3.887416362762451
avg loss per token: 0.030644760971469294;		runtime: 3.8610825538635254
['DT', 'NN', 'MD', 'VB', 'CD', 'RB', 'NN', 'PUNC']
[('the', 'DT'), ('flight', 'NN'), ('should', 'MD'), ('be', 'VB'), ('eleven', 'CD'), ('a.m', 'RB'), ('tomorrow', 'NN'), ('.', 'PUNC')]
['PRP', 'MD', 'VB', 'PRP', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'PRP', 'MD', 'VB', 'DT', 'NN', 'WDT', 'VBZ', 'NN', 'PUNC']
[('i', 'PRP'), ('would', 'MD'), ('like', 'VB'), ('it', 'PRP'), ('to', 'TO'), ('have', 'VB'), ('a', 'DT'), ('stop', 'NN'), ('in', 'IN'), ('new', 'NNP'), ('york', 'NNP'), ('and', 'C

'TODO: (reference HW1 part3.py)\n\t  * Use read_pos_file() from utils.py to read train.pos, val.pos, and test.pos.\n\t  * Initialize the model with training data, embedding dim 128, and hidden dim 256.\n\t  * Train the model, calling fit(), on the training data.\n\t  * Test the model, calling evaluate(), on the validation and test data.\n\t  * Predict outputs for the first ten examples in test.pos.\n\t  * Remove all instances of `raise NotImplementedError`!'

In [ ]:
# from google.colab import files
# files.download("my_model.pt")

['DT', 'NN', 'MD', 'VB', 'IN', 'CD', 'RB', 'NN', 'PUNC']

[('the', 'DT'), ('flight', 'NN'), ('should', 'MD'), ('arrive', 'VB'), ('at', 'IN'), ('eleven', 'CD'), ('a.m', 'RB'), ('tomorrow', 'NN'), ('.', 'PUNC')]



['PRP', 'MD', 'VB', 'TO', 'VB', 'DT', 'NN', 'WDT', 'VBZ', 'IN', 'NNP', 'NNP', 'NNP', 'TO', 'NNP', 'NNP', 'PUNC']

[('i', 'PRP'), ('would', 'MD'), ('like', 'VB'), ('to', 'TO'), ('find', 'VB'), ('a', 'DT'), ('flight', 'NN'), ('that', 'WDT'), ('goes', 'VBZ'), ('from', 'IN'), ('la', 'NNP'), ('guardia', 'NNP'), ('airport', 'NN'), ('to', 'TO'), ('san', 'NNP'), ('jose', 'NNP'), ('.', 'PUNC')]



['VB', 'PRP', 'DT', 'NNS', 'IN', 'NNP', 'TO', 'NNP', 'NNP', 'PUNC']

[('show', 'VB'), ('me', 'PRP'), ('the', 'DT'), ('flights', 'NNS'), ('from', 'IN'), ('newark', 'NNP'), ('to', 'TO'), ('los', 'NNP'), ('angeles', 'NNP'), ('.', 'PUNC')]



['WDT', 'NN', 'VBZ', 'DT', 'PUNC']

[('what', 'WDT'), ('airline', 'NN'), ('is', 'VBZ'), ('this', 'DT'), ('?', 'PUNC')]



['VB', 'PRP', 'DT', 'NNP', 'NNP', 'NNP', 'NN', 'PUNC']

[('show', 'VB'), ('me', 'PRP'), ('the', 'DT'), ('t', 'NNP'), ('w', 'NNP'), ('a', 'NNP'), ('flight', 'NN'), ('.', 'PUNC')]



['PRP', 'MD', 'VB', 'TO', 'VB', 'TO', 'NNP', 'PUNC']

[('i', 'PRP'), ('would', 'MD'), ('like', 'VB'), ('to', 'TO'), ('travel', 'VB'), ('to', 'TO'), ('westchester', 'NNP'), ('.', 'PUNC')]



['VB', 'NNP', 'NNP', 'NNS', 'IN', 'NNP', 'NNP', 'NNP', 'TO', 'NNP', 'PUNC']

[('list', 'VB'), ('american', 'NNP'), ('airlines', 'NNP'), ('flights', 'NNS'), ('from', 'IN'), ('new', 'NNP'), ('york', 'NNP'), ('newark', 'NNP'), ('to', 'TO'), ('nashville', 'NNP'), ('.', 'PUNC')]



['UH', 'PUNC']

[('thanks', 'UH'), ('.', 'PUNC')]



['WDT', 'NNS', 'VBP', 'EX', 'IN', 'NNP', 'TO', 'NNP', 'NN', 'NN', 'WDT', 'VBP', 'NN', 'PUNC']

[('what', 'WDT'), ('flights', 'NNS'), ('are', 'VBP'), ('there', 'EX'), ('from', 'IN'), ('nashville', 'NNP'), ('to', 'TO'), ('houston', 'NNP'), ('tomorrow', 'NN'), ('evening', 'NN'), ('that', 'WDT'), ('serve', 'VBP'), ('dinner', 'NN'), ('?', 'PUNC')]




['PRP', 'MD', 'VB', 'TO', 'VB', 'JJ', 'NNP', 'PUNC']

[('i', 'PRP'), ("'d", 'MD'), ('like', 'VB'), ('to', 'TO'), ('fly', 'VB'), ('next', 'JJ'), ('friday', 'NNP'), ('.', 'PUNC')]


**[2 points]** Free response: For the first 10 sentences of test.pos, **report the POS tags predicted by your model.**



- What works well, and what doesn't?
> It seems like the words are being properly tagged with only one error (that I spotted) amoung the first. However place names seem to be an area of struggle, and it has to be noted that it seems most of these sentences are to the point.

- For words tagged incorrectly, why do you think it happens, and what tag do they tend to get?
>

One word that was mis-tagged was airport going from NN to NNP. In this it seems that they give the tage that most closely resembling the original. In this case it was NN (general noun) turning into NNP.

- Think about micro- and macro-level tags (e.g., is it tagging NNS as NN, or VBD as NN, and which one is worse?).
> It seems mistagging NN is higher, as thats the one that I spotted. Hoever NNS to NN isn't too bad as it's under the same general catergory of noun. However VBD to NN is much worse, as it's under a completely different meaning.


## Part 2: Probabilistic Context Free Grammar
In this part, you will learn a PCFG given training data annotated with parse trees.

In [11]:
import trees
import fileinput
import collections
import re
"""You should not need any other imports, but you may import anything that helps."""

counts = collections.defaultdict(collections.Counter)

**[4 points]** Write code to read in trees and to count all the rules used in each tree. **Terminal nodes should be POS tags rather than words** to allow us to use the POS tagger. This means the grammar must contain a set of rules mapping each nonterminal POS tags in the PTB tag set to a terminal POS tag:
```
DT -> DT_t
NN -> NN_t
NNS -> NNS_t
```


#I HATE TREES!!!!

In [12]:
def count_rules():

  """TODO: Collect all the tree branching rules used in the parses in train.trees, and count their frequencies.
    * Use trees.Tree.from_str(), bottomup(), and other helpful functions from trees.py.
    * Goal: end up with three dictionaries, counts, probs, and cfg.
      * counts has entries count[LHS][RHS], like count[NP][(DT, NN)].
      * probs has entries prob[LHS][RHS] = count[LHS][RHS] / sum(count[LHS].values())
      * cfg simply has the rules of the grammar, stored using whichever structure is usable to you for your CKY implementation. For instance, indexing by RHS may be easier to look up for CKY (cfg[RHS][LHS].
      * To include terminal words as POS_t (e.g., NN_t) as you're constructing the CFG:
        if len(node.children) == 1: # terminal rules
          rhs = (node.label.split('_')[-1]+'_t',) # rhs = (NN_t,)
  """

  rule_counts = collections.defaultdict(lambda: collections.defaultdict(int)) # {rule_n : count}
  rule_probability = collections.defaultdict(dict) # {rule_n : prob}
  rule_cfg = collections.defaultdict(list) # {rule_n : cfg}


  #for line in fileinput.input("/content/nlp_hw2/data/train.trees"):
  with open("/content/nlp_hw2/data/train.trees", "r") as f:
    #lines = f.read()
    #print(lines) #<-- all trees
    for line in f:
      #print(line)
      if line is None:
        continue
      tree = trees.Tree.from_str(line) # one tree
      #print(tree)
      #if tree.root is None: #done with this tree will move on
      #  continue
      #print("scary")
      for node in tree.bottomup(): #go to the bottom of the rules
        if len(node.children) == 0 : # may need to just check if its zero
          continue
        lhs = node.label
        if len(node.children) == 1: # terminal rules
          rhs = (node.label.split('_')[-1]+'_t',)
        else:
          rhs = tuple(child.label for child in node.children)

        rule_counts[lhs][rhs] += 1

  #now for probabilites
  #lhs_counts = collections.defaultdict(dict)
  for lhs in rule_counts:
      total = sum(rule_counts[lhs].values())
      #lhs_counts[lhs] += count
      for rhs in rule_counts[lhs]:
        rule_probability[lhs][rhs] = rule_counts[lhs][rhs] / total
        #rule_cfg[lhs].append(rhs)

  #finally cfg
  for lhs, counting in rule_counts.items():
    for rhs in counting:
      rule_cfg[rhs].append(lhs)

  return rule_counts, rule_probability, rule_cfg
  #raise NotImplementedError

**[3 points]** Write code to compute the conditional probability of each rule and **print the PCFG in a readable format**, such as:
```
NP -> DT NN # 0.5
NP -> DT NNS # 0.5
```

In [13]:
def print_pcfg():
  #computing the conditional probabaility of each rule
  #print("begining")
  rule_counts, rule_probability, rule_cfg = count_rules()
  #print(rule_counts)
  #print()
  #print(rule_probability)
  #print()
  #print(rule_cfg)

  for lhs in rule_probability:
    for rhs in rule_probability[lhs]:
      print(f"{lhs} -> {' '.join(rhs)} # {rule_probability[lhs][rhs]}")

  #get unique rules
  unique_rules = set()
  for lhs in rule_cfg:
    for rhs in rule_cfg[lhs]:
      unique_rules.add((lhs, rhs))
  #print number of uniqe rules
  print(f"Number of unique rules: {len(unique_rules)}")

  #top 10 most frequent rules
# Top 10 most frequent rules (LHS -> RHS)
  all_rules_with_counts = [((lhs, rhs), count)
                          for lhs in rule_counts
                          for rhs, count in rule_counts[lhs].items()]

  top_10_rules = sorted(all_rules_with_counts, key=lambda x: x[1], reverse=True)[:10]

  print("\nTop 10 Most Frequent Rules:")
  for (lhs, rhs), count in top_10_rules:
      rhs_str = ' '.join(rhs)
      print(f"{lhs} -> {rhs_str} # Count: {count}")




  #top 6 highest-probability rules with left hand side NP
  top_6_rules = sorted(rule_probability['NP'].items(), key=lambda x: x[1], reverse=True)[:6]
  for rule in top_6_rules:
    print(f"{rule[0]} # {rule[1]}")
    #print(f"{rule[0]} -> {' '.join(rule[1])} # {sum(rule[1].values())}")



  #printing pcfg in readable format
  #raise NotImplementedError

**[3 points]** Running the code on train.trees, **report**:
- How many unique rules are there?
> 419
- What are the top five most frequent rules, and how many times did each occur?
> IN -> IN_t # Count: 482
>
>PUNC -> PUNC_t # Count: 469
>
>NP_NNP -> NNP_t # Count: 451
>
>NNP -> NNP_t # Count: 408
>
>NN -> NN_t # Count: 281
- What are the top five highest-probability rules with left-hand side NP, and what are their probabilities?
> ('NNP', 'NNP') # 0.1927536231884058
>
>('NP', 'NP*') # 0.11594202898550725
>
>('DT', 'NN') # 0.10144927536231885
>
>('DT', 'NNS') # 0.08550724637681159
>
>('DT', 'NP*') # 0.08405797101449275
- **Free Response**: Did the most frequent rules surprise you? Why or why not?
> Not really becuase nouns are one of the most frequent used word 'type' so I'm not surprise that they ended up frequenting it as much as they did


In [90]:
def main():
  print_pcfg()
main()

VB -> VB_t # 1.0
DT -> DT_t # 1.0
NNS -> NNS_t # 1.0
NP -> DT NNS # 0.08550724637681159
NP -> NP NP* # 0.11594202898550725
NP -> DT NN # 0.10144927536231885
NP -> NP PP # 0.04057971014492753
NP -> CD NNS # 0.002898550724637681
NP -> NNP NNP # 0.1927536231884058
NP -> NP_NNP NP # 0.013043478260869565
NP -> NP SBAR # 0.01884057971014493
NP -> NP_CD PP # 0.0014492753623188406
NP -> DT JJ # 0.007246376811594203
NP -> NNP NP* # 0.0463768115942029
NP -> QP RB # 0.004347826086956522
NP -> JJ NNP # 0.004347826086956522
NP -> DT NP* # 0.08405797101449275
NP -> CD RB # 0.030434782608695653
NP -> PDT NP* # 0.004347826086956522
NP -> NP VP # 0.005797101449275362
NP -> CD NP* # 0.02318840579710145
NP -> DT NX # 0.0014492753623188406
NP -> NN NN # 0.013043478260869565
NP -> NP_NNP NP_NNP # 0.007246376811594203
NP -> NN NP* # 0.017391304347826087
NP -> NNP JJ # 0.002898550724637681
NP -> NNP POS # 0.002898550724637681
NP -> NN NNS # 0.010144927536231883
NP -> NP_NN NP* # 0.007246376811594203
NP -> NP

# Part 3: CKY Parsing
In this part, you will implement the CKY parsing algorithm.

**[4 points]** Implement a CKY parser that takes your grammar and a set of POS tags as input, and outputs the highest-probability parse. If you can't find any parse, output a blank line. Use **log-probabilities** to avoid underflow.

In [120]:
def parse_cky():
  #Grammar and Probabilites
    #cfg / counts / probs: representations of your PCFG
  rule_counts, prob_rules, cfg_rules = count_rules()

  #true POS tags or POS tags as the output of a trained BiLSTM POS tagger
  train = read_pos_file("/content/nlp_hw2/data/train.pos")

  tag_vocab = set()
  for sentence in train:
    for item in sentence:
      tag_vocab.add(item[1])
  tag_vocab.add("<UNK>")

  #POS Tagging + diagonal initialization
  model = BiLSTMTagger(train, embedding_dim=128, hidden_dim=256)
  model.fit(train, lr=0.01, epochs=5)

  test = read_pos_file("/content/nlp_hw2/data/test.pos")

  for sent_num, sent in enumerate(test[:10]):
    words = [word.lower() for word, _ in sent]

    n = len(words)
    if n == 0:
      print()
      continue

    print(words)
    #predicted_tags = model.predict(words) #get predicted values
    predicted_tags = [t if t in tag_vocab else "" for w, t in sent]
    #print(predicted_tags)
    #predicted_tags = assign_tags(words, model)
    #predicted_tags = [t if t in tag_vocab else "<UNK>" for w, t in sent]
    #print(predicted_tags)


    chart = collections.defaultdict(dict)
    backptr = collections.defaultdict(dict)

    #Diagonal initialization (length-1 spans [i, i+1)):
    for i,tag in enumerate(predicted_tags):
      terminal_tag = tag + "t"
      chart[(i, i+1)][terminal_tag] = 0.0
      backptr[(i, i+1)][terminal_tag] = words[i]

    #CKY dynsmaic program over spans (big nested loop)
    for span_length in range(2, n+1):
      for i in range(n-span_length+1):
        k = i + span_length
        for j in range(i+1, k):

          for left_label in chart[(i, j)]:
            for right_label in chart[(j, k)]:
              # Check if any rule A -> left_label right_label exists in your PCFG
              rhs = (left_label, right_label) #<-- may need to come back and check
              if rhs in cfg_rules:
                for A in cfg_rules[left_label, right_label]:
                  candidate_score = chart[(i, j)][left_label] + chart[(j, k)][right_label] + math.log(prob_rules[rhs]) #<-- again MAY NEED TO CHECK IF RHS WORKS
                  #chart[(i, j)][B] + chart[(j, k)][C] + math.log(prob)

                  if A not in chart[(i, k)] or candidate_score > chart[(i, k)][A]:
                    chart[(i, k)][A] = candidate_score
                    backptr[(i, k)][A] = (left_label, j, right_label)

    #3) ROOT SELECTION, RECONSTRUCTION, PRINTING
    def reconstruct(label, i, k):
      bp = backptr[(i, k)][label]
      if isinstance(bp, str):
        return f"({label} {bp})"
      else:
        left_label, j, right_label = bp
        left_subtree = reconstruct(left_label, i, j)
        right_subtree = reconstruct(right_label, j, k)
        return f"({label} {left_subtree} {right_subtree})"

    if 'TOP' in chart[(0, n)]:
      tree = reconstruct('TOP', 0, n)
      print(tree)
      print(f"{chart[(0, n)]['TOP']:.4f}")
    else:
      print()


  #Root selection + backpointer reconstruction +printing




  """
  Goal: Highest-probability parse using a PCFG, with POS tags (from your BiLSTM)
        as the terminal layer.

  Inputs:
    - cfg / counts / probs: representations of your PCFG
    - true POS tags or POS tags as the output of a trained BiLSTM POS tagger

  Output:
    - For each sentence line from stdin or a file, print one bracketed tree to stdout
      (or an empty line if no parse) with its probability.

  Keep these three phases conceptually separate:
    (1) POS tagging + diagonal initialization
    (2) CKY dynamic program over spans (big nested loop)
    (3) Root selection + backpointer reconstruction + printing

  You are free to choose your exact data structures, as long as you can:
    - store best scores for labels over spans
    - remember how each best item was built (backpointers)
    - reconstruct a bracketed tree string at the end

  -----------------------------------------------------------------------------
  0) GRAMMAR + PROBABILITIES + <unk>
  -----------------------------------------------------------------------------
  * When reading a sequence of POS tags, map each tag to itself if in vocab,
    or else to a special token like "<unk>" (but keep original for printing).
  * After reconstruction, print the original word as the leaf instead.
  * Use log probabilities to avoid underflow:
    score = log P(A -> B C) + score(left) + score(right)

  -----------------------------------------------------------------------------
  1) READ ONE SENTENCE LINE → TOKENS → POS TAGS → DIAGONAL INIT
  -----------------------------------------------------------------------------
  - Read lines from train.pos as (word, tag) tuples using read_pos_files().
  - Run your BiLSTM POS tagger on the words to get one POS per token.
    * Hint: to confirm that CKY works, first just use the true POS tags rather than running it through your tagger.

  - Create two core tables for CKY (choose your own structures, examples below):
      chart:   stores best scores for labels over spans
              indexable by span (i, k) and then by label
      backptr: stores how that best label@span was formed
              (for terminals: the terminal tag; for binary: (left_label, split_index, right_label))

    Example shapes (you can pick others):
      chart[(i, k)][label]  -> best_score (log-prob or prob)
      backptr[(i, k)][label] -> for terminal: stored tag
                                for binary: (left_label, j, right_label)

  - Diagonal initialization (length-1 spans [i, i+1)):
      For each position i:
        * Use the POS tag(s) for token i as candidate preterminals.
        * Record best scores per POS at (i, i+1).
        * Record backptr so reconstruction can print "(POS word)".

  -----------------------------------------------------------------------------
  2) CKY DYNAMIC PROGRAM (THE BIG NESTED LOOP)
  -----------------------------------------------------------------------------
  The standard CKY fill uses three nested loops over span length, start index,
  and split point. Conceptually:

    for span_length in 2..n:
      for i in 0..(n - span_length):
        k = i + span_length
        initialize chart[(i, k)] and backptr[(i, k)] (empty)

        for j in (i+1)..(k-1):   # split index
          # Consider all ways to combine a left piece (i, j) with a right piece (j, k)
          for each left_label in chart[(i, j)]:
            for each right_label in chart[(j, k)]:
              # Check if any rule A -> left_label right_label exists in your PCFG
              for each A with P(A -> left_label right_label):
                candidate_score = chart[(i, j)][left_label] + \
                                  chart[(j, k)][right_label] + \
                                  log P(A -> left_label right_label)
                if candidate_score is better than current chart[(i, k)][A]:
                    update chart[(i, k)][A] = candidate_score
                    set backptr[(i, k)][A] = (left_label, j, right_label)

  Notes:
    - Only binary rules are considered here (CNF).
    - Keep everything in log-space abd use addition rather than multiplication.

  -----------------------------------------------------------------------------
  3) ROOT SELECTION, RECONSTRUCTION, PRINTING
  -----------------------------------------------------------------------------
  - After the table is filled, focus on the full span (0, n).
    * Prefer the designated start symbol (e.g., 'TOP') if present at (0, n).
    * If 'TOP' is not present, produce an empty parse.

  - Reconstruct the tree via backpointers:
    * Define a recursive function:
        reconstruct(label, i, k):
          bp = backptr[(i, k)][label]
          if bp is a terminal word (or "<unk>"):
              return "(label word_or_original)"
          else:
              (left_label, j, right_label) = bp
              left_subtree  = reconstruct(left_label,  i, j)
              right_subtree = reconstruct(right_label, j, k)
              return f"(label {left_subtree} {right_subtree})"

    * Ensure terminals print the original word here rather than POS tags.

  - Output:
    * Print the bracketed tree string for each input sentence (or an empty line
      if no parse), one sentence per line.
    * Print the final score for TOP at (0, n) as a log-prob.

  -----------------------------------------------------------------------------
  4) PRACTICAL TIPS / DECISIONS (YOU CHOOSE)
  -----------------------------------------------------------------------------
  - Data structures:
      * dict-of-dicts is fine; you can also use defaultdicts.
      * You can index chart/backptr by tuples (i, k) or use a 2D list.

  - Efficiency:
      * Iterate only over labels that actually occur in the subspans.
      * If your PCFG is stored by RHS (B, C) -> {A: prob}, you can quickly find
        candidate parents A for a given pair (B, C).

  - Scores:
      * Prefer log-space: add logs instead of multiplying probabilities.

  - Debugging:
      * Print the diagonal cells after initialization to verify POS entries.
      * For a tiny sentence (2–3 words), print intermediate chart cells per length.
      * If reconstruction fails, check that backptr entries are actually written
        whenever you write a score.

  -----------------------------------------------------------------------------
  5) MINIMUM I/O LOOP
  -----------------------------------------------------------------------------
  for each line from stdin:
    tokens = line.split()
    tags = run_pos_tagger(orig_tokens)

    initialize empty chart/backptr

    # diagonal init
    for i in range(n):
      fill chart[(i, i+1)] and backptr[(i, i+1)] with POS entries

    # CKY nested loops (length, start i, split j) using binary PCFG rules
    fill chart/backptr for spans of length >= 2

    if TOP in chart[(0, n)]:
        tree_str = reconstruct('TOP', 0, n)   # bracketed
        print(tree_str)
        print(logprob_of_TOP_to_stderr)
    else:
        print("")  # empty line if no parse
  """
  #raise NotImplementedError()

**[1 point]** Test the CKY parser with the test set with **gold POS tags**. **Print** the parse trees for the **first 10 test sentences** (use empty line for no parse). For full credit, these should match or be similar to the true parses in test.trees.

In [122]:
def test_parser():
  # Load grammar counts, probabilities, and cfg rules
  rule_counts, prob_rules, cfg_rules = count_rules()

  # Load test data with gold tags
  train = read_pos_file("/content/nlp_hw2/data/train.pos")
  test = read_pos_file("/content/nlp_hw2/data/test.pos")

  tag_vocabualry = set()
  for sentence in test:
    for item in sentence:
      tag_vocabualry.add(item[1])
  tag_vocabualry.add("<UNK>")


  for sent_num, sent in enumerate(test[:10]):
    words = [word.lower() for word, _ in sent]
    tags = [t if t in tag_vocabualry else "" for w, t in sent]
    print(words)
    print(tags)

    n = len(words)
    if n == 0:
        continue
    chart = collections.defaultdict(dict)
    backptr = collections.defaultdict(dict)

    # Diagonal initialization using gold tags
    for i, tag in enumerate(tags): #tag of predicted tags
        terminal_tag = tag + "t"
        chart[(i, i+1)][terminal_tag] = 0.0  # log(1)
        backptr[(i, i+1)][terminal_tag] = words[i]

    # CKY dynamic programming
    for span_length in range(2, n+1):
      for i in range(n - span_length + 1):
        k = i + span_length
        for j in range(i+1, k):
          for left_label in chart[(i, j)]:
            for right_label in chart[(j, k)]:
              rhs = (left_label, right_label)
              if rhs in cfg_rules:
                for A in cfg_rules[rhs]:
                  prob = prob_rules.get((A, rhs), 0.0)
                  if prob == 0:
                    continue
                  candidate_score = (
                      chart[(i, j)][left_label] +
                      chart[(j, k)][right_label] +
                      math.log(prob)
                  )
                  if A not in chart[(i, k)] or candidate_score > chart[(i, k)][A]:
                      chart[(i, k)][A] = candidate_score
                      backptr[(i, k)][A] = (left_label, j, right_label)




    def reconstruct(label, i, k):
      bp = backptr[(i, k)][label]
      if isinstance(bp, str):
        return f"({label} {bp})"
      else:
        left_label, j, right_label = bp
        left_subtree = reconstruct(left_label, i, j)
        right_subtree = reconstruct(right_label, j, k)
        return f"({label} {left_subtree} {right_subtree})"

    if 'TOP' in chart[(0, n)]:
      tree = reconstruct('TOP', 0, n)
      print(tree)
      #print(f"{chart[(0, n)]['TOP']:.4f}")
    else:
      print()


  #raise NotImplementedError()

test_parser()
print("First 10 Lines")
with open("/content/nlp_hw2/data/test.trees", "r") as file:
  lines = file.readlines()
  for line in lines[:10]:
    print(line.strip())
    #readable format





['the', 'flight', 'should', 'arrive', 'at', 'eleven', 'a.m', 'tomorrow', '.']
['DT', 'NN', 'MD', 'VB', 'IN', 'CD', 'RB', 'NN', 'PUNC']

['i', 'would', 'like', 'to', 'find', 'a', 'flight', 'that', 'goes', 'from', 'la', 'guardia', 'airport', 'to', 'san', 'jose', '.']
['PRP', 'MD', 'VB', 'TO', 'VB', 'DT', 'NN', 'WDT', 'VBZ', 'IN', 'NNP', 'NNP', 'NN', 'TO', 'NNP', 'NNP', 'PUNC']

['show', 'me', 'the', 'flights', 'from', 'newark', 'to', 'los', 'angeles', '.']
['VB', 'PRP', 'DT', 'NNS', 'IN', 'NNP', 'TO', 'NNP', 'NNP', 'PUNC']

['what', 'airline', 'is', 'this', '?']
['WDT', 'NN', 'VBZ', 'DT', 'PUNC']

['show', 'me', 'the', 't', 'w', 'a', 'flight', '.']
['VB', 'PRP', 'DT', 'NNP', 'NNP', 'NNP', 'NN', 'PUNC']

['i', 'would', 'like', 'to', 'travel', 'to', 'westchester', '.']
['PRP', 'MD', 'VB', 'TO', 'VB', 'TO', 'NNP', 'PUNC']

['list', 'american', 'airlines', 'flights', 'from', 'new', 'york', 'newark', 'to', 'nashville', '.']
['VB', 'NNP', 'NNP', 'NNS', 'IN', 'NNP', 'NNP', 'NNP', 'TO', 'NNP', '

**[2 points]** Integrate your **Bi-LSTM tagger** to assign POS tags to the words of any input sentence before passing it in to your CKY parser.

In [110]:
def assign_tags(sentence, model):

  print("begining")
  predicted = []
  print(sentence)

  #model = BiLSTMTagger(sentence, embedding_dim=128, hidden_dim=256) <-- has been initialized outside the function
  model.fit(sentence, lr=0.01, epochs=5)

  #assign POS tags to words in a sentence:
  #for word in sentence:

  for sent in sentence:
    words = [word.lower() for word, tag in sent]
    idxs = [model.words.numberize(word.lower()) for word, tag in sent] # prepped like before
    num_sequence = torch.tensor (idxs, dtype=torch.long)

    eval = model(num_sequence)
    pre = model.predict(eval)
    predicted.append(pre)

  print(predicted)


  # Return word-tag pairs
  print("end")
  #return list(zip(palabras, predicted_tags))


  #raise NotImplementedError

**[1 point]** Test the full pipeline (**input → tagger → CKY**). **Print** the parse trees for the **first 10 test sentences** (use empty line for no parse) using the predicted POS tags.

In [121]:
def test_pipeline():
  sent = read_pos_file("/content/nlp_hw2/data/test.pos")
  model = BiLSTMTagger(sent, embedding_dim=128, hidden_dim=256)
  assign_tags(sent, model)

  parse_cky()

  #raise NotImplementedError()

test_pipeline()

begining
[[('the', 'DT'), ('flight', 'NN'), ('should', 'MD'), ('arrive', 'VB'), ('at', 'IN'), ('eleven', 'CD'), ('a.m', 'RB'), ('tomorrow', 'NN'), ('.', 'PUNC')], [('i', 'PRP'), ('would', 'MD'), ('like', 'VB'), ('to', 'TO'), ('find', 'VB'), ('a', 'DT'), ('flight', 'NN'), ('that', 'WDT'), ('goes', 'VBZ'), ('from', 'IN'), ('la', 'NNP'), ('guardia', 'NNP'), ('airport', 'NN'), ('to', 'TO'), ('san', 'NNP'), ('jose', 'NNP'), ('.', 'PUNC')], [('show', 'VB'), ('me', 'PRP'), ('the', 'DT'), ('flights', 'NNS'), ('from', 'IN'), ('newark', 'NNP'), ('to', 'TO'), ('los', 'NNP'), ('angeles', 'NNP'), ('.', 'PUNC')], [('what', 'WDT'), ('airline', 'NN'), ('is', 'VBZ'), ('this', 'DT'), ('?', 'PUNC')], [('show', 'VB'), ('me', 'PRP'), ('the', 'DT'), ('t', 'NNP'), ('w', 'NNP'), ('a', 'NNP'), ('flight', 'NN'), ('.', 'PUNC')], [('i', 'PRP'), ('would', 'MD'), ('like', 'VB'), ('to', 'TO'), ('travel', 'VB'), ('to', 'TO'), ('westchester', 'NNP'), ('.', 'PUNC')], [('list', 'VB'), ('american', 'NNP'), ('airlines', '

**[2 points]** Free response: For the first 10 sentences of test.pos…
- Is there a difference in which sentences it fails to parse given gold tags vs. your tagger's outputs? Why or why not?
>  It's seems to do words once the taggers output is added. But there is a high (very likely) possibility that this is a result from the way I implemented it (was having many issues)

- Which ones does it do well on (i.e., match the true parse in test.trees), and why?
> It seem's to do well overall since the words are pretty common and the sentences themselves aren't complicated in structure and grammatically.
- Which ones does it do poorly on (but still produces a parse), and why?
> Some of the issues my come from past problems in other functions (this was quite difficult so I wouldn't be surprised if that was the case). But tis seems for me at least, it would make mistaks on similar tags.


**Congratulations! That's a wrap on HW2. Onto neural methods and greater generalizability.**